In [1]:
import torch
import pandas as pd
from utils.mbert import MBert
from utils.metrics import get_BLEU

/home/filip/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-02 21:19:02.595686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748891942.744439    5823 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748891942.784863    5823 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748891943.083262    5823 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

target_lan = "pl_PL"
csv_path = "data/en_pl.csv"

# target_lan = "de_DE"
# csv_path = "data/en_de.csv"

cuda


In [3]:
src_sentences = [
    "Hello, how are you doing today?",
    "The meeting will start in ten minutes.",
    "Can you help me translate this sentence?"
]
ref_sentences = [
    "Cześć, jak się dzisiaj masz?",
    "Spotkanie zacznie się za dziesięć minut.",
    "Czy możesz pomóc mi przetłumaczyć to zdanie?"
]

df_temp = pd.DataFrame({
    "english": src_sentences,
    "polish": ref_sentences
})
df_temp

,english,polish
0,"Hello, how are you doing today?","Cześć, jak się dzisiaj masz?"
1,The meeting will start in ten minutes.,Spotkanie zacznie się za dziesięć minut.
2,Can you help me translate this sentence?,Czy możesz pomóc mi przetłumaczyć to zdanie?


In [4]:
chunk_size = 100_000

file_reader = pd.read_csv(csv_path, chunksize=chunk_size)
first_chunk = next(file_reader) 
print(type(first_chunk))
print(first_chunk.head())

<class 'pandas.core.frame.DataFrame'>
                                english  \
0      Previously on "The Blacklist"...   
1        - You want to call your daddy?   
2  - Yeah, I want to tell him I'm okay.   
3                                 Okay.   
4  Lizzy... Be careful of your husband.   

                                   polish  
0             /W poprzednich odcinkach: /  
1             - Chcesz zadzwonić do taty?  
2  - Tak, powiem, że wszystko w porządku.  
3                                 Dobrze.  
4                  Lizzy, uważaj na męża.  


In [4]:
mbert = MBert(device, target_lan)

In [5]:
translations = mbert.translate(df_temp['english'])
ref = df_temp['polish'].to_list()
bleu = get_BLEU(translations, [ref])

Tłumaczenie: 100%|██████████| 3/3 [02:00<00:00, 40.00s/it] 


BLEU score: 41.44


In [ ]:
translated_lists = []
for df in file_reader:
    translations = mbert.translate(df['english'])
    translated_lists.append(translations)
    

Tłumaczenie:   2%|▏         | 2429/100000 [16:56<25:11:36,  1.08it/s]

In [ ]:
bleus = []
for refs, trans in zip(file_reader, translated_lists):
    refs = refs['polish'].to_list()
    bleu = get_BLEU(trans, [refs])
    bleus.append(bleu)

In [ ]:
import pickle

with open("data/translations_mbart.pkl", "wb") as f:
    pickle.dump(translated_lists, f)

with open("data/bleus.pkl", "wb") as f:
    pickle.dump(bleus, f)